In [1]:
#EASY

import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, make_scorer

data = pd.read_csv("../data/bmw.csv")
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


In [2]:
data = data.drop(["model", "transmission", "fuelType"], axis=1)
data.head()

,year,price,mileage,tax,mpg,engineSize
0,2014,11200,67068,125,57.6,2.0
1,2018,27000,14827,145,42.8,2.0
2,2016,16000,62794,160,51.4,3.0
3,2017,12750,26676,145,72.4,1.5
4,2014,14500,39554,160,50.4,3.0


In [3]:
X = data.drop("price", axis=1)
y = data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(r2)
pd.DataFrame(model.feature_importances_,X.columns)

0.8924211422653983


,0
year,0.456929
mileage,0.102855
tax,0.018154
mpg,0.150389
engineSize,0.271673


In [4]:
#АНАЛИЗ

# r2 = 0.89, хороший показатель
# наиболее важные признаки - год выпуска и объем двигателя, это логично
# Дальше идут потребление топлива и пробег. потребление топлива важнее налога тк на дистанции очевидно вносит больший процент трат.

In [5]:
#NORMAL

df = pd.DataFrame()
for file in os.listdir("../data"):
    if 'unclean' not in file and file.endswith(".csv"): # unclean прям очень плохие данные, проще их не брать
        data = pd.read_csv(f"../data/{file}")
        if file == "hyundi.csv":
            data = data.rename(columns={"tax(£)": "tax"})
        df = pd.concat([df, data], axis=0, ignore_index=True)

df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0


In [6]:
df.fillna(0, inplace=True)
df = pd.get_dummies(df, columns=["model", "transmission", "fuelType"], drop_first=True)
df = df.sample(int(len(df)/10), random_state=42) # очень долго обучается моделька, поэтому чуть пожертвуем количество данных

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("price", axis=1), df["price"], test_size=0.2, random_state=42)

model = RandomForestRegressor()

params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 20, 30, 40, 50],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

In [8]:
r2_scorer = make_scorer(r2_score)
search = RandomizedSearchCV(model, params, n_iter=10, scoring=r2_scorer, n_jobs=-1, cv=5, random_state=42)
search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring=make_scorer(r2_score))

In [9]:
search.best_params_

{'n_estimators': 300,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_depth': 50}

In [12]:
y_pred = search.predict(X_test)

r2 = r2_score(y_test, y_pred)
r2

0.9220415938154302

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
search_RFC = RandomizedSearchCV(model, params, n_iter=10, scoring="accuracy", n_jobs=-1, cv=5, random_state=42)
search_RFC.fit(X_train, y_train)

C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
14 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py", line 473, in fit
    trees = Paralle

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='accuracy')

In [14]:
y_pred_RFC = search_RFC.predict(X_test)
r2_RFC = r2_score(y_test, y_pred_RFC)

r2_RFC

0.8259300784886336

In [16]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

params_DTC = {
    "max_depth": [10, 20, 30, 40, 50],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

search_DTC = RandomizedSearchCV(model, params_DTC, n_iter=10, scoring="accuracy", n_jobs=-1, cv=5, random_state=42)
search_DTC.fit(X_train, y_train)

C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10]},
                   random_state=42, scoring='accuracy')

In [17]:
y_pred_DTC = search_DTC.predict(X_test)
r2_DTC = r2_score(y_test, y_pred_DTC)

r2_DTC

0.7921748327400678

In [18]:
#АНАЛИЗ

# Все абсолютно закономерно, рандом регрессор лучше всех и дальше по убыванию